In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
import numpy as np
import pandas as pd

In [3]:
%pip install  

Note: you may need to restart the kernel to use updated packages.


ERROR: You must give at least one requirement to install (see "pip help install")


In [4]:
from simpletransformers.classification import ClassificationModel

c:\Users\TOSHIBA\anaconda3\envs\core\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import glob

In [24]:
path = "D:\\AKINSOFT\\aysenurakpinar"
filenames = glob.glob(path + "\*.xlsx")
print('File names:', filenames)
for file in filenames:
    a = pd.concat(pd.read_excel( file, sheet_name=None), ignore_index=True, sort=False)

File names: ['D:\\AKINSOFT\\aysenurakpinar\\Projem.xlsx']


In [25]:
#a=pd.read_excel("/content/drive/MyDrive/Bert/Projem.xlsx")

In [26]:
a

,kategori,metin
0,Antropoloji,Şu anda güneybatı Çin olarak tanımlanan coğraf...
1,Antropoloji,"Arkeologlar, Avrupa’nın ilk Homo sapiens’inin ..."
2,Antropoloji,"81 yaşındaki, Sosyoloji Profesörü Orlando Patt..."
3,Antropoloji,"Yeni bir araştırmaya göre, Güneydoğu Avrupa'nı..."
4,Antropoloji,"1721 yılında Norveç'li bir misyoner, Viking at..."
...,...,...
5695,Yerbilimleri,"Görsel açıklaması: Uluslararası bilim ekibi, O..."
5696,Yerbilimleri,Antarktika’da incelemeler yapan buzul bilimci ...
5697,Yerbilimleri,Anadolu yer sincabının coğrafi dağılımının içi...
5698,Yerbilimleri,21 Ağustos 1999 tarihinde Hürriyet gazetesinde...


In [27]:
a["kategori"].unique()

array(['Antropoloji', 'Arkeoloji', 'Astronomi', 'Bilişim', 'Biyoloji',
       'Çevre Bilimleri', 'Evrim', 'Felsefe', 'Fizik', 'Kimya',
       'Matematik', 'Nöroloji', 'Bilim Ve Teknoloji', 'Ekonomi', 'Sağlık',
       'Siyaset', 'Spor', 'Psikoloji', 'Sağlık Bilimleri', 'Sosyoloji',
       'Tarih', 'Teknoloji', 'Yerbilimleri'], dtype=object)

In [28]:
a['labels'] = pd.factorize(a.kategori)[0]

In [29]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(a, test_size=0.2, random_state=42)

In [30]:
train=train[["metin","labels"]]
test=test[["metin","labels"]]

In [31]:
#for bert text = string      label = int
train["metin"]=train["metin"].apply(lambda r: str(r))
train['labels']=train['labels'].astype(int)

In [32]:
model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-uncased', num_labels=23, use_cuda=False,
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 3, "train_batch_size": 64 , "fp16":False, "output_dir": "bert_model"})

Some weights of the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

In [33]:
model.train_model(train)

c:\Users\TOSHIBA\anaconda3\envs\core\lib\site-packages\simpletransformers\classification\classification_model.py:601: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
  0%|          | 10/4560 [00:11<1:28:31,  1.17s/it]
c:\Users\TOSHIBA\anaconda3\envs\core\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 3 of 3: 100%|██████████| 3/3 [4:24:39<00:00, 5293.18s/it]  


(216, 1.297636457339481)

In [34]:
result, model_outputs, wrong_predictions = model.eval_model(test)

c:\Users\TOSHIBA\anaconda3\envs\core\lib\site-packages\simpletransformers\classification\classification_model.py:1442: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
Running Evaluation: 100%|██████████| 143/143 [06:38<00:00,  2.79s/it]


In [36]:
predictions = model_outputs.argmax(axis=1)

In [37]:
actuals = test.labels.values

In [38]:
predictions[:10]

array([ 5,  6, 16,  2, 13,  2, 11, 10, 13, 16], dtype=int64)

In [39]:
actuals[:10]

array([ 8,  4, 16,  8, 13,  5,  9, 10, 13, 16], dtype=int64)

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(actuals, predictions)

0.706140350877193

In [41]:
örnek = test.iloc[43]['metin']
print(örnek)

“Çoğu zaman şiddetin ne kadar yakınımızda olduğunun farkına bile varmayız, çünkü ondan kaçınmak için tavizler vermişizdir.”_x000D_
_x000D_
Julian Assange_x000D_
_x000D_
Dinleme skandalı gündemden düşmüyor. Geçtiğimiz hafta Alman basınında...


In [42]:
tahmin=model.predict([örnek])

if tahmin[0] ==0:
  print("ANTROPOLOJİ")
elif tahmin[0]==1:
  print("ARKEOLOJİ")
elif tahmin[0]==2:
  print("ASTRONOMİ")
elif tahmin[0]==3:
  print("BİLİŞİM")
elif tahmin[0]==4:
  print("BİYOLOJİ")
elif tahmin[0]==5:
  print("ÇEVRE BİLİMLERİ")
elif tahmin[0]==6:
  print("EVRİM")
elif tahmin[0]==7:
  print("FELSEFE") 
elif tahmin[0]==8:
  print("FİZİK") 
elif tahmin[0]==9:
  print("KİMYA") 
elif tahmin[0]==10:
  print("MATEMATİK") 
elif tahmin[0]==11:
  print("NÖROLOJİ") 
elif tahmin[0]==12:
  print("BİLİM VE TEKNOLOJİ") 
elif tahmin[0]==13:
  print("EKONOMİ") 
elif tahmin[0]==14:
  print("SAĞLIK") 
elif tahmin[0]==15:
  print("SİYASET") 
elif tahmin[0]==16:
  print("SPOR") 
elif tahmin[0]==17:
  print("PSİKOLOJİ") 
elif tahmin[0]==18:
  print("SAĞLIK BİLİMLERİ") 
elif tahmin[0]==19:
  print("SOSYOLOJİ") 
elif tahmin[0]==20:
  print("TARİH") 
elif tahmin[0]==21:
  print("TEKNOLOJİ")
else:
  print("YERBİLİMLERİ")

100%|██████████| 1/1 [00:00<00:00,  2.57it/s]

TARİH


In [43]:
#deneme import
#denemem=pd.read_excel("/content/drive/MyDrive/Bert/denemem.xlsx",names=["kategori","metin"])

In [48]:
path = "D:\\AKINSOFT\\aysenurakpinar\\denemeversi"
filenames = glob.glob(path + "\*.xlsx")
print('File names:', filenames)
for file in filenames:
    denemem = pd.concat(pd.read_excel( file, sheet_name=None), ignore_index=True, sort=False)

File names: ['D:\\AKINSOFT\\aysenurakpinar\\denemeversi\\denemem.xlsx']


In [49]:
denemem

,kategori,metin
0,Spor,İngiltere Premier Lig'in 33. haftasında Liverp...
1,Siyaset,Çanakkale İl Genel Meclisi Başkanı ve AK Parti...
2,Siyaset,Ana muhalefet partisi CHP’nin 12 yıldır başarı...
3,Siyaset,CHP’nin seçimden seçime seçmenini hatırladığın...
4,Spor,Premier Lig'de 2017-2018 ve 2018-2019 sezonlar...
5,Spor,Süper Lig'de 37. hafta mücadelesinde Konyaspor...
6,Ekonomi,ABD Ticaret Bakanı Gina Raimondo da iklim deği...
7,Ekonomi,BD'nin ev sahipliğinde 40 ülke liderinin katıl...
8,Ekonomi,"Artık birçok işletme, modellerinde gölge karbo..."
9,Teknoloji,Apple dün gerçekleştirdiği etkinlikte M1 işlem...


In [50]:
dene=denemem.iloc[3]["metin"]

In [51]:
örnek=denemem.iloc[11]["metin"]

In [52]:
tahmin=model.predict([örnek])

if tahmin[0] ==0:
  print("ANTROPOLOJİ")
elif tahmin[0]==1:
  print("ARKEOLOJİ")
elif tahmin[0]==2:
  print("ASTRONOMİ")
elif tahmin[0]==3:
  print("BİLİŞİM")
elif tahmin[0]==4:
  print("BİYOLOJİ")
elif tahmin[0]==5:
  print("ÇEVRE BİLİMLERİ")
elif tahmin[0]==6:
  print("EVRİM")
elif tahmin[0]==7:
  print("FELSEFE") 
elif tahmin[0]==8:
  print("FİZİK") 
elif tahmin[0]==9:
  print("KİMYA") 
elif tahmin[0]==10:
  print("MATEMATİK") 
elif tahmin[0]==11:
  print("NÖROLOJİ") 
elif tahmin[0]==12:
  print("BİLİM VE TEKNOLOJİ") 
elif tahmin[0]==13:
  print("EKONOMİ") 
elif tahmin[0]==14:
  print("SAĞLIK") 
elif tahmin[0]==15:
  print("SİYASET") 
elif tahmin[0]==16:
  print("SPOR") 
elif tahmin[0]==17:
  print("PSİKOLOJİ") 
elif tahmin[0]==18:
  print("SAĞLIK BİLİMLERİ") 
elif tahmin[0]==19:
  print("SOSYOLOJİ") 
elif tahmin[0]==20:
  print("TARİH") 
elif tahmin[0]==21:
  print("TEKNOLOJİ")
else:
  print("YERBİLİMLERİ")

100%|██████████| 1/1 [00:00<00:00,  2.93it/s]

SAĞLIK


In [55]:
import pickle
filename = 'D:\\AKINSOFT\\aysenurakpinar\\aysenurakpinarmodel\\finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))